In [17]:
import numpy as np
import pandas as pd

In [18]:
# Mapping the Week date
WeekMapDF = pd.read_excel("IRI week translation_2008_2017.xls", usecols=["IRI Week","Calendar week starting on"])
WeekMapDF.columns = ["WEEK","WEEK_DATE"]

WeekMapDF["MONTH"] = WeekMapDF["WEEK_DATE"].dt.month
season_map = {
        12: "Winter", 1: "Winter", 2: "Winter",
        3: "Spring", 4: "Spring", 5: "Spring",
        6: "Summer", 7: "Summer", 8: "Summer",
        9: "Fall", 10: "Fall", 11: "Fall"
    }
WeekMapDF["SEASON"] = WeekMapDF["MONTH"].map(season_map)

# Holiday flag
holiday_weeks = [
    ('2008-12-29', 'New Year'),
    ('2009-01-26', 'Super Bowl'),
    ('2009-02-09', 'Valentine'),
    ('2009-03-16', 'St Patrick'),
    ('2009-04-06', 'Easter'),
    ('2009-05-25', 'Memorial Day'),
    ('2009-06-29', 'Independence Day'),
    ('2009-09-07', 'Labor Day'),
    ('2009-10-26', 'Halloween'),
    ('2009-11-23', 'Thanksgiving'),
    ('2009-12-21', 'Christmas')
]
holidays_2009 = pd.DataFrame(holiday_weeks, columns=['WEEK_DATE', 'HOLIDAY'])
holidays_2009['WEEK_DATE'] = pd.to_datetime(holidays_2009['WEEK_DATE'])
holidays_2009['IS_HOLIDAY'] = 1
WeekMapDF = WeekMapDF.merge(holidays_2009, on='WEEK_DATE', how='left')
WeekMapDF['IS_HOLIDAY'] = WeekMapDF['IS_HOLIDAY'].fillna(0).astype(int)

WeekMapDF["WEEK_DATE"] = pd.to_datetime(WeekMapDF["WEEK_DATE"])
WeekMapDF.head()

,WEEK,WEEK_DATE,MONTH,SEASON,HOLIDAY,IS_HOLIDAY
0,1479,2007-12-31,12,Winter,NaN,0
1,1480,2008-01-07,1,Winter,NaN,0
2,1481,2008-01-14,1,Winter,NaN,0
3,1482,2008-01-21,1,Winter,NaN,0
4,1483,2008-01-28,1,Winter,NaN,0


In [19]:
"""STORE DETAILS"""
StoresDF = pd.read_excel("Delivery_Stores_9.xlsx")

# Filtered 'EAU CLAIRE' & 'OU' stores
StoresDF = StoresDF[(StoresDF['OU'] == 'GR') & (StoresDF['Market_Name'] == 'EAU CLAIRE')]
StoresDF = StoresDF.drop(columns=['OU','Market_Name','MskdName','Open','Clsd'])

StoresDF = StoresDF.reset_index(drop=True)
StoresDF

,IRI_KEY,EST_ACV
0,228037,11.85
1,233779,27.22899
2,250845,29.98
3,257871,39.95898
4,264075,29.98
5,653776,46.49899
6,1085053,71.45697


In [20]:
"""BEER DETAILS"""
BeerDF = pd.read_excel("prod11_beer.xlsx")

# Create PACKAGE_OZ
BeerDF['PACKAGE_OZ'] = BeerDF['L9'].str.split().str[-1].str.replace('OZ', '', regex=True).astype(float)
BeerDF = BeerDF.drop(columns=['L9'])

# Remove '-' from UPC
BeerDF['UPC'] = BeerDF['UPC'].apply(lambda x: int(str(x).replace("-", "").strip()))
BeerDF = BeerDF.drop(columns=['SY','GE','VEND','ITEM','L1','Level'])
BeerDF = BeerDF.rename(columns={'L2': 'CATEGORY', 'L3': 'PARENT_CO', 'L4': 'VENDOR', 'L5': 'BRAND',
                                'PRODUCT TYPE': 'PRODUCT_TYPE', 'TYPE OF BEER/ALE': 'TYPE_OF_BEER',
                                'FLAVOR/SCENT': 'FLAVOR_SCENT', 'CALORIE LEVEL': 'CALORIE_LEVEL'})

# Drop rows with missing value
BeerDF = BeerDF.dropna(subset=["FLAVOR_SCENT","SIZE","CALORIE_LEVEL","COLOR"])
BeerDF = BeerDF.drop(columns=['*AG C=1+ CATEGORY                                                        00004  '])

ouncedata = {
    "PACKAGE_OZ": [72.0238, 72.0, 40.0, 144.0, 24.0, 42.0, 12.0, 288.0, 216.0, 360.0,
                   240.0, 64.0, 56.0, 96.0, 67.1884, 16.0, 169.0, 67.2, 134.4, 59.6,
                   25.4, 11.2, 16.9, 17.0, 192.0, 69.0, 138.0, 168.0],
    "UNIT_OZ": [12.0, 12.0, 40.0, 12.0, 24.0, 7.0, 12.0, 12.0, 12.0, 12.0,
                     12.0, 16.0, 7.0, 16.0, 67.1884, 16.0, 169.0, 67.2, 11.2, 59.6,
                     25.4, 11.2, 16.9, 17.0, 12.0, 11.5, 11.5, 12.0]
            }
ounceDF = pd.DataFrame(ouncedata)
BeerDF = BeerDF.merge(ounceDF, on='PACKAGE_OZ', how='left')

BeerDF = BeerDF.reset_index(drop=True)
BeerDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16758 entries, 0 to 16757
Data columns (total 15 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   CATEGORY       16758 non-null  object 
 1   PARENT_CO      16758 non-null  object 
 2   VENDOR         16758 non-null  object 
 3   BRAND          16758 non-null  object 
 4   UPC            16758 non-null  int64  
 5   VOL_EQ         16758 non-null  float64
 6   PRODUCT_TYPE   16758 non-null  object 
 7   TYPE_OF_BEER   16758 non-null  object 
 8   PACKAGE        16758 non-null  object 
 9   FLAVOR_SCENT   16758 non-null  object 
 10  SIZE           16758 non-null  object 
 11  CALORIE_LEVEL  16758 non-null  object 
 12  COLOR          16758 non-null  object 
 13  PACKAGE_OZ     16758 non-null  float64
 14  UNIT_OZ        13420 non-null  float64
dtypes: float64(3), int64(1), object(11)
memory usage: 1.9+ MB


In [21]:
"""CUSTOMER DEMOGRAPHICS"""
CustDemographicDF=pd.read_csv("ads demos9.CSV")
CustDemographicDF = CustDemographicDF[['Panelist ID', 'Panelist Type', 
       'Combined Pre-Tax Income of HH', 'Family Size', 'Type of Residential Possession',
       'Age Group Applied to Household Head', 'Education Level Reached by Household Head', 'Occupation Code of Household Head',
       'Age Group Applied to Male HH', 'Education Level Reached by Male HH', 'Occupation Code of Male HH',
       'Age Group Applied to Female HH', 'Education Level Reached by Female HH', 'Occupation Code of Female HH',
       'Number of Dogs','Number of Cats', 'Children Group Code', 'Marital Status', 'RACE3']]

CustDemographicDF = CustDemographicDF.replace('.', 0)

replacements = {
    'Combined Pre-Tax Income of HH': {'99': 0},
    'Age Group Applied to Household Head': {'99': 0}, 'Age Group Applied to Male HH': {'99': 0}, 'Age Group Applied to Female HH': {'99': 0},
    'Education Level Reached by Household Head': {'99': 0}, 'Education Level Reached by Male HH': {99: 0}, 'Education Level Reached by Female HH': {99: 0},
    'Occupation Code of Household Head': {'98': 0, '99': 0}, 'Occupation Code of Male HH': {99: 0}, 'Occupation Code of Female HH': {98: 0, 99: 0},
    'Marital Status': {99: 0}, 'RACE3': {99: 0}
}

for col, rep in replacements.items():
    CustDemographicDF[col] = CustDemographicDF[col].replace(rep).astype(int)

rename_map = {
    'Panelist ID': 'PANID',
    'Panelist Type': 'PAN_TYPE',
    'Combined Pre-Tax Income of HH': 'INCOME',
    'Family Size': 'FAMILY_SIZE',
    'Type of Residential Possession': 'RESIDENCE_TYPE',
    'Age Group Applied to Household Head': 'AGE_HH',
    'Education Level Reached by Household Head': 'EDU_HH',
    'Occupation Code of Household Head': 'OCU_HH',
    'Age Group Applied to Male HH': 'AGE_MHH',
    'Education Level Reached by Male HH': 'EDU_MHH',
    'Occupation Code of Male HH': 'OCU_MHH',
    'Age Group Applied to Female HH': 'AGE_FHH',
    'Education Level Reached by Female HH': 'EDU_FHH',
    'Occupation Code of Female HH': 'OCU_FHH',
    'Number of Dogs': 'DOGS',
    'Number of Cats': 'CATS',
    'Children Group Code': 'CHILDREN',
    'Marital Status': 'MARITAL_STATUS',
    'RACE3': 'RACE'
}
CustDemographicDF = CustDemographicDF.rename(columns=rename_map)

CustDemographicDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4607 entries, 0 to 4606
Data columns (total 19 columns):
 #   Column          Non-Null Count  Dtype
---  ------          --------------  -----
 0   PANID           4607 non-null   int64
 1   PAN_TYPE        4607 non-null   int64
 2   INCOME          4607 non-null   int64
 3   FAMILY_SIZE     4607 non-null   int64
 4   RESIDENCE_TYPE  4607 non-null   int64
 5   AGE_HH          4607 non-null   int64
 6   EDU_HH          4607 non-null   int64
 7   OCU_HH          4607 non-null   int64
 8   AGE_MHH         4607 non-null   int64
 9   EDU_MHH         4607 non-null   int64
 10  OCU_MHH         4607 non-null   int64
 11  AGE_FHH         4607 non-null   int64
 12  EDU_FHH         4607 non-null   int64
 13  OCU_FHH         4607 non-null   int64
 14  DOGS            4607 non-null   int64
 15  CATS            4607 non-null   int64
 16  CHILDREN        4607 non-null   int64
 17  MARITAL_STATUS  4607 non-null   int64
 18  RACE            4607 non-nul

In [22]:
"""CUSTOMER TRANSACTIONS"""
CustTransactionsDF = pd.read_csv("trips9 may13.csv")
CustTransactionsDF['TRIP_COST'] = CustTransactionsDF['KRYSCENTS']/100
CustTransactionsDF = CustTransactionsDF.drop(columns=['IRI_Key2','CENTS998','CENTS999','KRYSCENTS'])

# Filter based on IRI_KEY
CustTransactionsDF = CustTransactionsDF[(CustTransactionsDF['IRI_KEY'].isin(StoresDF['IRI_KEY']))]

# Mapping Week date
CustTransactionsDF = CustTransactionsDF.merge(WeekMapDF[['WEEK','WEEK_DATE']], on='WEEK', how='left')

CustTransactionsDF = CustTransactionsDF.reset_index(drop=True)
CustTransactionsDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 228689 entries, 0 to 228688
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype         
---  ------     --------------   -----         
 0   PANID      228689 non-null  int64         
 1   WEEK       228689 non-null  int64         
 2   IRI_KEY    228689 non-null  int64         
 3   MINUTE     228689 non-null  int64         
 4   TRIP_COST  228689 non-null  float64       
 5   WEEK_DATE  228689 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(4)
memory usage: 10.5 MB


In [ ]:
"""STORE TRANSACTIONS"""
StoreTransactionsDF=pd.read_excel("BeerEnvECPF2009.xlsx")
StoreTransactionsDF = StoreTransactionsDF.rename(columns={'DOLLARS': 'PURCHASE_PRICE', 'F': 'FEATURE', 'D': 'DISPLAY', 'PR': 'PRICE_REDUCTION'})

# Filter EAU CLAIRE stores and based on IRI_KEY
StoreTransactionsDF=StoreTransactionsDF[(StoreTransactionsDF['Market_Name'] == 'EAU CLAIRE')]
StoreTransactionsDF=StoreTransactionsDF[(StoreTransactionsDF['IRI_KEY'].isin(StoresDF['IRI_KEY']))]

# Create UPC column
StoreTransactionsDF["UPC"] = (StoreTransactionsDF['SY'] * 100000000000 + StoreTransactionsDF['@GE'] * 10000000000 + StoreTransactionsDF['VEND'] * 100000 + StoreTransactionsDF['ITEM']).astype(int)
StoreTransactionsDF = StoreTransactionsDF.drop(columns=['SY','@GE','VEND','ITEM'])
StoreTransactionsDF = StoreTransactionsDF.drop(columns=['Market_Name'])

# Create Unit price column
StoreTransactionsDF["UNIT_PRICE"] = StoreTransactionsDF["PURCHASE_PRICE"] / StoreTransactionsDF["UNITS"]

# Create PROMO = 1 if any of FEATURE, DISPLAY, PRICE_REDUCTION was used.
StoreTransactionsDF['FEATURE'] = (StoreTransactionsDF['FEATURE'] != 'NONE').astype(int)
StoreTransactionsDF['DISPLAY'] = (StoreTransactionsDF['DISPLAY'] > 0).astype(int)
StoreTransactionsDF['PROMO'] = (
    (StoreTransactionsDF['FEATURE'] > 0) |
    (StoreTransactionsDF['DISPLAY'] > 0) |
    (StoreTransactionsDF['PRICE_REDUCTION'] > 0)
).astype(int)

# Handle rows with 0 Units
print(len(StoreTransactionsDF[(StoreTransactionsDF['UNITS'] == 0)]),'rows with 0 Units \n')
StoreTransactionsDF = StoreTransactionsDF.loc[(StoreTransactionsDF["UNITS"]  > 0)]

# Mapping Week date
StoreTransactionsDF = StoreTransactionsDF.merge(WeekMapDF[['WEEK','WEEK_DATE']], on='WEEK', how='left')

StoreTransactionsDF = StoreTransactionsDF.reset_index(drop=True)
StoreTransactionsDF.info()

81 rows with 0 Units 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109616 entries, 0 to 109615
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype         
---  ------           --------------   -----         
 0   IRI_KEY          109616 non-null  int64         
 1   WEEK             109616 non-null  int64         
 2   UNITS            109616 non-null  int64         
 3   PURCHASE_PRICE   109616 non-null  float64       
 4   FEATURE          109616 non-null  int64         
 5   DISPLAY          109616 non-null  int64         
 6   PRICE_REDUCTION  109616 non-null  int64         
 7   UPC              109616 non-null  int64         
 8   UNIT_PRICE       109616 non-null  float64       
 9   PROMO            109616 non-null  int64         
 10  WEEK_DATE        109616 non-null  datetime64[ns]
dtypes: datetime64[ns](1), float64(2), int64(8)
memory usage: 9.2 MB


In [ ]:
"""BEER TRANSACTIONS"""
BeerTransactionsDF = pd.read_csv("beer_PANEL_GK_1531_1582.DAT", delimiter=",")
BeerTransactionsDF = BeerTransactionsDF.rename(columns={'COLUPC': 'UPC', 'DOLLARS': 'BEER_PRICE', 'UNITS': 'BEER_UNITS'}).drop(columns=['OUTLET'])

# Filter based on IRI_KEY
BeerTransactionsDF = BeerTransactionsDF[(BeerTransactionsDF['IRI_KEY'].isin(StoresDF['IRI_KEY']))]
print('Unique IRI_KEYs: ',BeerTransactionsDF['IRI_KEY'].unique())

# {0:'Mon', 1:'Tue', 2:'Wed', 3:'Thu', 4:'Fri', 5:'Sat', 6:'Sun'}
BeerTransactionsDF['DAY_OF_WEEK'] = (BeerTransactionsDF['MINUTE'] // 1440).astype(int)
BeerTransactionsDF['HOUR'] = (BeerTransactionsDF['MINUTE'] % 1440 // 60).astype(int)

# Merge "FEATURE","DISPLAY","PRICE_REDUCTION" & Beer Details
BeerTransactionsDF = (BeerTransactionsDF
    .merge(BeerDF, on="UPC", how="inner")
    .merge(WeekMapDF[['WEEK','WEEK_DATE','IS_HOLIDAY']], on="WEEK", how="left")
    )

# Create UNIT_PRICE & UNIT_PRICE_OZ
BeerTransactionsDF["PRICE_PER_UNIT"] = BeerTransactionsDF["BEER_PRICE"] / BeerTransactionsDF["BEER_UNITS"]

BeerTransactionsDF["PRICE_PER_OZ"] = BeerTransactionsDF["PRICE_PER_UNIT"] / BeerTransactionsDF["UNIT_OZ"]
BeerTransactionsDF['OZ'] = BeerTransactionsDF['UNIT_OZ'] * BeerTransactionsDF['BEER_UNITS']
BeerTransactionsDF['VOL_EQ'] = BeerTransactionsDF['BEER_UNITS'] * BeerTransactionsDF['VOL_EQ'] 

# Handle empty values
BeerTransactionsDF = BeerTransactionsDF.dropna()
#BeerTransactionsDF = BeerTransactionsDF.drop(columns='WEEK')
BeerTransactionsDF['BEER_PURCHASED'] = 1

BeerTransactionsDF = BeerTransactionsDF.groupby('PANID').filter(lambda x: len(x) >= 2)
unique_panids = BeerTransactionsDF['PANID'].unique()
'''rng = np.random.default_rng(seed=7)

customers_to_include = rng.choice(unique_panids, size=400, replace=False)
BeerTransactionsDF = BeerTransactionsDF[BeerTransactionsDF['PANID'].isin(customers_to_include)]'''

CustTransactionsDF = CustTransactionsDF[CustTransactionsDF['PANID'].isin(unique_panids)]

BeerTransactionsDF = BeerTransactionsDF.reset_index(drop=True)
BeerTransactionsDF.info()

Unique IRI_KEYs:  [ 257871 1085053]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5424 entries, 0 to 5423
Data columns (total 29 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   PANID           5424 non-null   int64         
 1   WEEK            5424 non-null   int64         
 2   MINUTE          5424 non-null   int64         
 3   BEER_UNITS      5424 non-null   float64       
 4   BEER_PRICE      5424 non-null   float64       
 5   IRI_KEY         5424 non-null   int64         
 6   UPC             5424 non-null   int64         
 7   DAY_OF_WEEK     5424 non-null   int64         
 8   HOUR            5424 non-null   int64         
 9   CATEGORY        5424 non-null   object        
 10  PARENT_CO       5424 non-null   object        
 11  VENDOR          5424 non-null   object        
 12  BRAND           5424 non-null   object        
 13  VOL_EQ          5424 non-null   float64       
 14  PRODUCT_TYPE    5424

In [25]:
print(StoreTransactionsDF.isna().any().any())
print(CustTransactionsDF.isna().any().any())
print(BeerTransactionsDF.isna().any().any())

False
False
False


In [26]:
WeekMapDF.to_excel('DFs\\WeekMapDF.xlsx', index=False)
StoresDF.to_excel('DFs\\StoresDF.xlsx', index=False)
BeerDF.to_excel('DFs\\BeerDF.xlsx', index=False)
CustDemographicDF.to_excel('DFs\\CustDemographicDF.xlsx', index=False)
StoreTransactionsDF.to_excel('DFs\\StoreTransactionsDF.xlsx', index=False)
CustTransactionsDF.to_excel('DFs\\CustTransactionsDF.xlsx', index=False)
BeerTransactionsDF.to_excel('DFs\\BeerTransactionsDF.xlsx', index=False)